In [78]:
%pip install flask tensorflow resampy librosa opencv-python pydub

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: flask in c:\users\usher\appdata\local\programs\python\python311\lib\site-packages (3.1.0)




[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from flask import Flask, render_template, request
import tensorflow as tf

In [ ]:
# Load the trained model
model = tf.keras.models.load_model('./model/cnn.keras')

In [3]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 256, 256, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 64, 64, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 64, 64, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 65536)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    33,554,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 67,398,998 (257.11 MB)

 Trainable params: 33,699,498 (128.55 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 33,699,500 (128.55 MB)

In [4]:
from numpy import ndarray
import numpy as np
import librosa
import matplotlib.pyplot as plt
import cv2 as cv
import io
import base64

HOP_LENGTH = 512        # number of samples between successive frames
WINDOW_LENGTH = 512     # length of the window in samples
N_MEL = 128             # number of Mel bands to generate


def spectrogram_fixed_length(audio, rate, total_samples = 128) -> ndarray:
    spectrogram = librosa.feature.melspectrogram(
        y = audio, 
        sr=rate, 
        hop_length=HOP_LENGTH, 
        win_length=WINDOW_LENGTH,
        n_fft=512
    )

    spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)

    spectrogram_length = spectrogram_db.shape[1]

    if spectrogram_length != total_samples:
        spectrogram_db = librosa.util.fix_length(
            spectrogram_db, size=total_samples, axis=1,

            #constant_values=(0, -80)
        )
    
    return spectrogram_db


def generate_spectrogram(audio, sample_rate):
    spectrogram = spectrogram_fixed_length(audio, sample_rate)
    librosa.display.specshow(spectrogram, cmap='viridis')
    plt.axis('tight')
    plt.axis('off')
    plt.tight_layout()
    plt.savefig("uploads/spectrogram.png", bbox_inches="tight", pad_inches=0.0)



# TODO: use sliding window
def preprocess_audio(audio):
    DURATION = 3
    audio, sample_rate = librosa.load(audio, duration=DURATION, res_type='kaiser_fast')
    generate_spectrogram(audio, sample_rate)
   
    # fig, ax = plt.subplots(figsize=(8, 6))
    # fig.axis('tight')
    # fig.axis('off')
    # fig.tight_layout()
    # spectrogram = librosa.display.spectrogram(spectrogram)
    # plt.close(fig)

    IMG_SIZE = 256

    img_array = cv.imread("uploads/spectrogram.png")
    new_array = cv.resize(img_array, (IMG_SIZE, IMG_SIZE))

    print("Done!")

    return np.array(new_array).reshape(-1, IMG_SIZE, IMG_SIZE, 3) / 255


In [ ]:
import os
from pydub import AudioSegment

app = Flask("Audio Classifier")
app.config['UPLOAD_FOLDER'] = 'uploads/'

@app.route('/')
def index():
    return render_template('./index.html')

# @app.route('/upload', methods=['POST'])
# def upload_file():
#     if 'file' not in request.files:
#         return 'No file uploaded', 400

#     file = request.files['file']

#     if file.filename == '':
#         return 'No file selected', 400

#     if file:
#         filename = file.filename
#         file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
#         generate_spectrogram(os.path.join(app.config['UPLOAD_FOLDER'], filename))
#         return 'File uploaded and spectrogram generated', 200

@app.route('/classify', methods=['POST'])
def classify():
    # Get the audio file from the request
    audio_file = request.files['audio_file']

    # Process the audio file using your trained model
    audio_data = preprocess_audio(audio_file)
    print("predicting...")
    prediction = model.predict(audio_data)

    
    LABELS = [
        "air_conditioner",
        "car_horn",
        "children_playing",
        "dog_bark",
        "drilling",
        "engine_idling",
        "gun_shot",
        "jackhammer",
        "siren",
        "street_music"
    ]

    p = {}
    pred = prediction.flatten()
    print(pred)
    for (index, label) in enumerate(LABELS):
        p[label] = float(pred[index] * 100.0)

    print(p)

    
    predicted = np.argmax(prediction)

    file = open("uploads/spectrogram.png", 'rb')
    image_buffer = file.read()

    print(audio_file.mimetype)
    spectrogram = base64.b64encode(image_buffer).decode('utf-8')


    # mp3_bytes = io.BytesIO()
    # audio = AudioSegment.from_file(audio_file.read())
    # audio.export(mp3_bytes, format='mp3')
    # # mp3_data = mp3_bytes.getvalue()
    
    # audio_data = base64.b64encode(mp3_bytes.read()).decode('utf-8')
    
    # Return the classification result
    return {
        'prediction': LABELS[predicted],
        'spectrogram': spectrogram, 
        # 'audio': audio_data,
        # 'audio_mime': audio_file.mimetype,
        'predictions': p,
    }

app.run()


 * Serving Flask app 'Audio Classifier'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


Done!
predicting...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step


127.0.0.1 - - [07/Apr/2025 16:02:12] "POST /classify HTTP/1.1" 200 -


[1.3643700e-13 1.7709755e-09 4.3041070e-04 6.7096233e-05 3.4395241e-06
 5.2609261e-07 4.1817664e-13 1.4929240e-06 1.9591214e-05 9.9947745e-01]
{'air_conditioner': 1.3643700427258543e-11, 'car_horn': 1.77097547737759e-07, 'children_playing': 0.043041069875471294, 'dog_bark': 0.006709623266942799, 'drilling': 0.0003439524107307079, 'engine_idling': 5.260926059236226e-05, 'gun_shot': 4.181766424023875e-11, 'jackhammer': 0.0001492923956902814, 'siren': 0.0019591214368119836, 'street_music': 99.94774460792542}
audio/mpeg


...

[5.4685844e-12 3.5691224e-07 1.6422293e-05 2.4885531e-03 1.1931844e-04
 8.6257011e-01 9.9069700e-13 1.6445614e-07 7.6578244e-06 1.3479739e-01]
{'air_conditioner': 5.468584352713579e-10, 'car_horn': 3.569122384305956e-05, 'children_playing': 0.0016422292901552282, 'dog_bark': 0.24885530583560467, 'drilling': 0.011931844346690923, 'engine_idling': 86.25701069831848, 'gun_shot': 9.906969992631853e-11, 'jackhammer': 1.6445613937321468e-05, 'siren': 0.000765782442613272, 'street_music': 13.479739427566528}
audio/x-flac
Done!
predicting...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step


127.0.0.1 - - [07/Apr/2025 16:18:21] "POST /classify HTTP/1.1" 200 -


[1.6689825e-09 1.7468348e-07 3.2726072e-02 9.4324368e-04 2.3821838e-05
 8.8182741e-01 4.1782819e-10 1.4653297e-04 3.2233614e-02 5.2099165e-02]
{'air_conditioner': 1.6689825077520481e-07, 'car_horn': 1.7468347834892484e-05, 'children_playing': 3.2726071774959564, 'dog_bark': 0.09432436781935394, 'drilling': 0.0023821838112780824, 'engine_idling': 88.18274140357971, 'gun_shot': 4.178281887146085e-08, 'jackhammer': 0.01465329696657136, 'siren': 3.223361447453499, 'street_music': 5.209916457533836}
application/ogg
Done!
predicting...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step


127.0.0.1 - - [07/Apr/2025 16:18:40] "POST /classify HTTP/1.1" 200 -


[1.0500895e-05 4.0726044e-04 1.7351497e-02 1.9714825e-01 3.1675138e-02
 6.6482621e-01 9.2407345e-06 1.8782335e-03 2.1550180e-02 6.5143429e-02]
{'air_conditioner': 0.0010500894859433174, 'car_horn': 0.040726043516770005, 'children_playing': 1.735149696469307, 'dog_bark': 19.714824855327606, 'drilling': 3.167513757944107, 'engine_idling': 66.48262143135071, 'gun_shot': 0.0009240734470949974, 'jackhammer': 0.1878233510069549, 'siren': 2.155018039047718, 'street_music': 6.514342874288559}
audio/wav


 * Serving Flask app 'Audio Classifier'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Apr/2025 14:43:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2025 14:44:03] "GET /?audio_file=siren.wav HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2025 14:44:33] "GET /?audio_file=silence.wav HTTP/1.1" 200 -
